In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import re
import time
import pdfplumber
from openpyxl import Workbook

In [ ]:

def generar_diccionario_ampliaciones(file):
    diccionario_obras_ampliacion = {}

    paginas = [1, 2]

    with pdfplumber.open(file) as pdf:
        text = ""
        for i in paginas:
            text += pdf.pages[i].extract_text()

        matches = re.finditer(r'((Ampliación en S/E).*?)(?=\n\S)', text, re.DOTALL)
        for match in matches:
            line = match.group(0)
            if not re.match(r".*[0-9]{2}$", line):
                print("Título largo: ", line)
                titulo = input("Ingrese el título del proyecto según el índice del pdf: ")
                pag_inicio = int(input("Ingrese la página de inicio del proyecto según el índice del pdf: ")) - 1
                pag_final = pag_inicio + 3

            else:
                match = re.match(r'^(.*?)\s+(\d+)$', line)
                if match:
                    titulo = match.group(1).replace(".", "")
                    pag_inicio = int(match.group(2)) - 1
                    pag_final = pag_inicio + 3

            
            diccionario_obras_ampliacion[titulo] = (pag_inicio, pag_final)


    return diccionario_obras_ampliacion




In [ ]:
def extraer_texto_entre_delimitadores_v2(texto, delimitador_inicial, delimitador_final):
    pattern = re.compile(rf"{re.escape(delimitador_inicial)}(.*?){re.escape(delimitador_final)}.*?\.", re.DOTALL)
    match = pattern.search(texto)
    return match.group(0) if match else "ERROR EXTRAYENDO TEXTO"


In [ ]:
from funciones_extra import generar_diccionario_ampliaciones, generar_diccionario_descripciones_amp, extraer_texto_entre_delimitadores_v2
from proyecto_ampliacion import Proyecto_ampliacion

In [ ]:
file = "plan_expansion_final_2023.pdf"
diccionario = generar_diccionario_ampliaciones(file)
dic_proyectos = generar_diccionario_descripciones_amp(file, diccionario)

In [ ]:

def extraer_texto(proyecto_str):
    # Definimos las frases de inicio y fin
    frase_inicio = "El proyecto consiste en"
    frase_fin = "El proyecto incluye todas las obras"
    
    # Utilizamos una expresión regular para encontrar el texto entre las frases
    patron = re.compile(re.escape(frase_inicio) + r'(.*?)(?=' + re.escape(frase_fin) + ')', re.DOTALL)
    
    # Buscamos el patrón en el string de entrada
    coincidencia = patron.search(proyecto_str)
    
    # Si encontramos una coincidencia, la devolvemos, de lo contrario devolvemos None
    if coincidencia:
        return coincidencia.group(0).strip()
    else:
        return None

In [ ]:
def ejecutar_analisis_v3(dic_proyectos):
    dic_proyectos_con_excepciones = {}
    for titulo, descripcion in dic_proyectos.items():
        proyecto = Proyecto_ampliacion(titulo, descripcion)
        proyecto.procesar_descripcion()


In [ ]:
ejecutar_analisis_v3(dic_proyectos)

In [ ]:
l_configs = ["barra principal seccionada y barra de transferencia", "interruptor y medio", "doble barra principal y barra de transferencia", "doble barra principal y barra de transferencia", "doble barra principal con barra de transferencia", "barra simple"]
l_configs2 = ["barra principal con barra de transferencia", "interruptor y medio", "barra principal seccionada y barra de transferencia", "barra simple", "doble barra principal y barra de transferencia", "barra principal más barra auxiliar", "barra simple seccionada", "barra principal más barra auxiliar", "barra principal y barra de transferencia"]
l_oficial = [['barra principal seccionada y barra de transferencia', 'interruptor y medio', 'doble barra principal y barra de transferencia', 'doble barra principal con barra de transferencia', 'barra simple', 'barra principal con barra de transferencia', 'barra principal más barra auxiliar', 'barra simple seccionada', 'barra principal y barra de transferencia']]

# vamos a recorrer ambas listas y agregar las configuraciones que no estén en la lista oficial
for config in l_configs:
    if config not in l_oficial:
        l_oficial.append(config)

for config in l_configs2:
    if config not in l_oficial:
        l_oficial.append(config)

print(l_oficial)


In [ ]:
# comprobacion de extraccion de configuraciones de patios

l_oficial = ['barra principal seccionada y barra de transferencia', 'interruptor y medio', 'doble barra principal y barra de transferencia', 'doble barra principal con barra de transferencia', 'barra simple', 'barra principal con barra de transferencia', 'barra principal más barra auxiliar', 'barra simple seccionada', 'barra principal y barra de transferencia']


textos = [
    "El proyecto consiste en la ampliación de las barras e instalaciones comunes del patio de 110 kV de la subestación Cóndores, cuya configuración corresponde a barra principal con barra de transferencia, para dos nuevas posiciones, de manera de permitir la conexión de la obra “Nueva S/E Alto Molle y nueva línea 2x110 kV Alto Molle – Cóndores”.",
    "El proyecto consiste en la ampliación del galpón e instalaciones comunes del patio de 220 kV de la subestación Nueva Cardones, cuya configuración corresponde a interruptor y medio, para cuatro nuevas diagonales, de manera de permitir la conexión de nuevos proyectos en la zona.",
    "El proyecto consiste en la ampliación del galpón e instalaciones comunes del patio de 220 kV de la subestación Nueva Maitencillo, cuya configuración corresponde a interruptor y medio, para cuatro nuevas diagonales, de manera de permitir la conexión de nuevos proyectos en la zona.",
    "El proyecto consiste en la ampliación de las barras principales e instalaciones comunes del patio de 220 kV de la subestación Nogales, cuya configuración corresponde a interruptor y medio, para dos nuevas diagonales, de manera de permitir la conexión de nuevos proyectos en la zona.",
    "El proyecto consiste en la ampliación de las barras principales e instalaciones comunes del patio de 220 kV de la subestación Santa Clara, cuya configuración corresponde a interruptor y medio, para dos nuevas diagonales, de manera de permitir la conexión de nuevos proyectos en la zona.",
    "El proyecto consiste en la ampliación de las barras principales e instalaciones comunes del patio de 220 kV de la subestación Rahue, cuya configuración corresponde a barra principal seccionada y barra de transferencia, para dos nuevas posiciones, de manera de permitir la conexión de nuevos proyectos en la zona.",
    "El proyecto consiste en la ampliación de la barra principal e instalaciones comunes del patio de 110 kV de la subestación Mejillones, cuya configuración corresponde a barra simple, para dos nuevas posiciones, de manera de permitir la conexión de la obra “Nueva S/E La Chimba y nueva línea 2x110 kV Mejillones – La Chimba”.",
    "El proyecto consiste en la ampliación de la barra e instalaciones comunes de la subestación La Portada, cuya configuración corresponde a barra simple, en una posición, de manera de permitir la conexión de la obra “Conexión de línea 1x110 kV Tap Desalant – Desalant en S/E La Portada”.",
    "El proyecto consiste en la ampliación de las barras e instalaciones comunes del patio de 110 kV de la subestación Guardiamarina, cuya configuración corresponde a doble barra principal y barra de transferencia, para dos nuevas posiciones, de manera de permitir la conexión de la obra “Nueva S/E Caracoles, nuevas líneas 2x220 kV Caracoles – Liqcau y 2x110 kV Guardiamarina – Caracoles”.",
    "El proyecto consiste en la ampliación del patio de 220 kV de la subestación Esmeralda, cuya configuración corresponde a barra simple, para pasar a una configuración de tipo interruptor y medio con espacio en barras y plataforma para al menos tres diagonales, de manera de permitir la conexión del banco de autotransformadores 220/110 kV existente, la conexión del paño asociado al proyecto “Extensión de línea 1x220 kV Atacama – Esmeralda y cambio de conexión a S/E Caracoles”, la conexión de la línea asociada al proyecto “Nueva línea 1x220 kV Esmeralda – Liqcau” y la conexión de nuevos proyectos en la zona. A su vez, el proyecto considera la conexión en 220 kV del banco de autotransformadores existente en la nueva configuración de barra. [CUANTAS POSICIONES QUEDAN DISP EN ESTE CASO??? CASO ESPECIAL]",
    "El proyecto consiste en la ampliación de las barras principales e instalaciones comunes del patio de 220 kV de la subestación Liqcau, antes denominada Nueva La Negra, cuya configuración corresponde a interruptor y medio, para tres nuevas diagonales, de manera de permitir la conexión de la obra “Nueva S/E Caracoles, nuevas líneas 2x220 kV Caracoles – Liqcau y 2x110 kV Guardiamarina – Caracoles”, la conexión de la obra “Nueva línea 1x220 kV Esmeralda – Liqcau” y la conexión de nuevos proyectos en la zona.",
    "A su vez, el proyecto contempla la construcción de una nueva barra de 110 kV, en configuración barra simple, la cual deberá considerar espacio en barra y plataforma para tres posiciones, de manera de permitir la conexión del nuevo transformador de poder antes mencionado, la conexión del transformador existente y la conexión del paño asociado a la obra “Adecuación de paño de línea 1x110 kV Esmeralda – Sur en S/E Sur”.",
    "Adicionalmente, el proyecto considera la ampliación de la sala de celdas de 13,8 kV, en celdas de configuración barra simple, contemplándose la construcción de, al menos, tres celdas para alimentadores, la celda de conexión del nuevo transformador antes mencionado, la construcción de una celda de interconexión con la barra existente y espacio para la instalación de cuatro paños futuros.",
    "Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 12 kV, en configuración barra principal más barra auxiliar, contemplándose la construcción de, al menos, cuatro paños para alimentadores, el paño de conexión del transformador antes mencionado a la barra principal, un paño de conexión del nuevo transformador a la barra auxiliar, la construcción de un paño de interconexión con la barra existente y espacio en barra para la construcción de cuatro paños futuros.",
    "Adicionalmente, el proyecto considera la construcción de una nueva sala de celdas de 12 kV, en configuración barra principal más barra auxiliar, contemplándose la construcción de, al menos, seis celdas para alimentadores, dos espacios para celdas de futuros alimentadores, las celdas de  conexión del transformador antes mencionado a la barra principal y a la barra auxiliar, la construcción de una celda para equipos de medida y la construcción de celdas para la interconexión con las barras de media tensión existentes.",
    "A su vez, el proyecto incluye la ampliación de la barra e instalaciones comunes del patio de 110 kV de la subestación, cuya configuración corresponde a barra simple seccionada, la cual deberá considerar espacio en barra y plataforma para una posición, de manera de permitir la conexión del nuevo transformador de poder antes mencionado. ",
    "Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 12 kV, en configuración barra principal más barra auxiliar, contemplándose la construcción de, al menos, cuatro paños para alimentadores, el paño de conexión del transformador antes mencionado, un paño para la conexión de un banco de condensadores, la construcción de un paño de interconexión con la barra existente y espacio en barra para la construcción de cuatro paños futuros. [CUANTAS POSICIONES QUEDAN DISP EN ESTE CASO??? CASO ESPECIAL]",
    "El proyecto consiste en la ampliación de la barra e instalaciones comunes del patio de 110 kV de la subestación Bollenar, cuya configuración corresponde a barra simple, para tres nuevas posiciones, de manera de permitir la conexión de un paño para la línea 1x110 kV Tap Off Alto Melipilla – Bollenar y la conexión de nuevos proyectos en la zona. ",
    "Adicionalmente, el proyecto contempla la construcción de un nuevo paño en configuración barra simple para conectar la línea 1x110 kV Tap Off Alto Melipilla – Bollenar en la subestación Bollenar.",
    "Además, el proyecto contempla la ampliación de la barra de 66 kV, en configuración barra simple, en una posición, de manera de permitir la normalización del paño de conexión del transformador de poder antes mencionado. ",
    "Adicionalmente, el proyecto considera la ampliación de la barra de 13,2 kV, en configuración barra simple, contemplándose la construcción de, al menos, 2 paños para alimentadores, el paño de conexión del nuevo transformador y espacio en barra para la construcción de tres paños futuros. [CUANTAS POSICIONES QUEDAN DISP EN ESTE CASO??? CASO ESPECIAL]",
    "El proyecto consiste en la ampliación de las barras e instalaciones comunes del patio de 66 kV de la subestación Fuentecilla, cuya configuración corresponde a barra principal con barra de transferencia, para tres nuevas posiciones, de manera de permitir la conexión de la línea asociada a la obra Nueva línea 2x66 kV Fuentecilla – El Carmen y un nuevo proyecto en la zona.",
    "A su vez, el proyecto considera la ampliación de la barra de 66 kV, en configuración barra simple, de manera de permitir el reemplazo del equipo de transformación antes mencionado, junto con el reemplazo de todo el equipamiento que se vea sobrepasado en sus características nominales producto del aumento de capacidad antes descrito. ",
    "Adicionalmente, el proyecto considera la ampliación de la barra de 13,8 kV, en configuración barra simple, contemplándose la construcción de, al menos, cuatro paños para alimentadores, el paño de conexión del transformador antes mencionado, la construcción de un paño de interconexión con la barra existente y un paño para la conexión de un banco de condensadores. [CUANTAS POSICIONES QUEDAN DISP EN ESTE CASO??? CASO ESPECIAL]",
    "El proyecto consiste en la ampliación de las barras e instalaciones comunes del patio de 66 kV de la subestación Monterrico, cuya configuración corresponde a barra principal con barra de transferencia, para dos nuevas posiciones, de manera de permitir la conexión de la obra “Nueva S/E Coihueco y nueva línea 2x66 kV Monterrico – Coihueco”.",
    "A su vez, el proyecto considera la ampliación de las barras de 66 kV, en configuración barra principal y barra de transferencia, de manera de permitir el reemplazo del equipo de transformación antes mencionado, junto con el reemplazo de todo el equipamiento que se vea sobrepasado en sus características nominales producto del aumento de capacidad antes descrito. ",
    "Adicionalmente, el proyecto considera la ampliación de la barra de 15 kV, en configuración barra simple, contemplándose la construcción de, al menos, cuatro paños para alimentadores, el paño de conexión del transformador antes mencionado y la construcción de un paño de interconexión con la barra existente.",
    "Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 23 kV, en configuración barra simple, contemplándose la construcción de, al menos, cuatro paños para alimentadores, el paño de conexión del transformador antes mencionado, la construcción de un paño de interconexión con la barra existente y espacio en barra para la construcción de dos paños futuros.",
    "A su vez, el proyecto incluye la ampliación de la barra e instalaciones comunes del patio de 66 kV de la subestación, cuya configuración corresponde a barra simple, la cual deberá considerar espacio en barra y plataforma para dos posiciones, de manera de permitir la conexión del nuevo transformador de poder antes mencionado y la normalización de la conexión de la línea 1x66 kV Tres Pinos – Cañete. ",
    "Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 23 kV, en configuración barra simple, contemplándose la construcción de, al menos, dos paños para alimentadores, el paño de conexión del transformador antes mencionado, un paño para la conexión de un banco de condensadores y la construcción de un paño de interconexión con la barra existente. [CUANTAS POSICIONES QUEDAN DISP EN ESTE CASO??? CASO ESPECIAL]",
    "Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 15 kV, en configuración barra simple, contemplándose la construcción de, al menos, tres paños para alimentadores, el paño de conexión del nuevo transformador 66/15 kV antes mencionado y la construcción de un paño de interconexión con la barra existente. ",
    "Además, el proyecto considera la construcción de una nueva sección de barra de 13,2 kV en configuración barra simple, contemplándose la construcción de, al menos, tres paños para alimentadores, un paño para para la reubicación del alimentador asociado al transformador T3 y el paño de conexión del transformador 66/13,2 kV.",
    "A su vez, el proyecto contempla la construcción de un nuevo patio de 66 kV, en configuración barra principal y barra de transferencia, con capacidad de barras de, al menos 300 MVA con 75°C en el conductor y 35°C temperatura ambiente con sol, donde se deberá considerar espacio en barras y plataforma para la construcción de cuatro posiciones, de manera de permitir la conexión del nuevo equipo de transformación y el existente, la construcción de un paño acoplador, la reubicación de la conexión de la línea 1x66 kV Aihuapi – Los Negros y espacio con terreno nivelado para al menos dos paños futuros para el seccionamiento de uno de los circuitos de la línea 2x66 kV Pilmaiquén – Osorno.",
    "Adicionalmente, el proyecto considera la construcción de una nueva barra de 23 kV, en configuración barra simple, contemplándose la construcción de, al menos, 3 paños para alimentadores, el paño de conexión del nuevo transformador y el paño para la conexión del transformador N°1."
    ]

trol = len(textos)
i = 0
for texto in textos:
    if any(config in texto for config in l_oficial):
        print("Texto válido")
        i +=1
    else:
        print("Texto inválido")
        print(texto)



print(i)
print(trol)

In [ ]:
import re


In [ ]:
# Comprobacion funcion de extraccion de tensiones de patios

def extraer_tension(texto):
    # vamos a buscar donde diga la frase "patio de X kV"
    pattern = re.compile(r'patio de \d+(?:,\d+)? kV')
    match = pattern.search(texto)

    if match:
        return match.group(0)
    
    else:
        # Vamos a buscar por tension de barras 
        pattern = re.compile(r'(barra|barras|sala de celdas|sección de barra|) de \d+(?:,\d+)? kV')
        match = pattern.search(texto)

        if match:
            return match.group(0)
        
        else:
            return "No se encontro tension de patio o barra o celda, por lo que se imprimira un resumen del proyecto directamente"
        

for texto in textos:
    tension = extraer_tension(texto)
    if tension:
        print(tension)
    else:
        print("No se encontró la tensión")
        print(texto)

In [ ]:
def extraer_numero_posiciones(texto):
    #vAMOS A RETORNAR EL NUMERO DE POSICIONES O DIAGONALES QUE TIENE EL PATIO COMO INT, PARA FACILITAR LUEGO EL CALCULO DE POSICIONES DISPONIBLES
    numeros = {
        "uno": 1,
        "una": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9,
        "diez": 10
    }

    # Patrones de búsqueda para posiciones, diagonales y paños
    patron_posiciones_diagonales = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (posiciones|diagonales)\b', re.IGNORECASE)
    patron_panos = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (paño|paños)\b', re.IGNORECASE)

    # Buscar coincidencias en el texto
    match_posiciones_diagonales = patron_posiciones_diagonales.search(texto)
    match_panos = patron_panos.search(texto)

    if match_posiciones_diagonales:
        numero_str, tipo = match_posiciones_diagonales.groups()
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)

        if tipo.lower() == "diagonales":
            return numero * 2
        elif tipo.lower() == "posiciones":
            return numero
        
    elif match_panos:
        numero_str = match_panos.group(1)
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)
        return f"al menos {numero} paño(s)"

    else:
        return False 


def extraer_numero_posiciones_v2(texto):
    numeros = {
        "uno": 1,
        "una": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9,
        "diez": 10
        }

    # Patrones de búsqueda para posiciones, diagonales y paños
    patron_posiciones_diagonales = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez)(?:( nueva| nuevas))? (posición|posiciones|diagonales)\b', re.IGNORECASE)
    patron_panos = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (paño|paños)\b', re.IGNORECASE)


    # Buscar coincidencias en el texto
    match_posiciones_diagonales = patron_posiciones_diagonales.search(texto)
    match_panos = patron_panos.search(texto)
    
    if match_posiciones_diagonales:
        numero_str, _, tipo = match_posiciones_diagonales.groups()
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)

        if tipo.lower() == "diagonales":
            return numero * 2
        elif tipo.lower() == "posiciones":
            return numero
        
    elif match_panos:
        numero_str = match_panos.group(1)
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)
        return f"al menos {numero} paño(s)"
    
    else:
        return False

def extraer_numero_posiciones_v3(texto):
    numeros = {
        "uno": 1,
        "una": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9,
        "diez": 10
    }
    pattern1 = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez)(?: (nueva|nuevas))? (posición|posiciones|diagonal|diagonales)\b', re.IGNORECASE)
    pattern2 = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (celda para alimentador|celdas para alimentadores)\b', re.IGNORECASE)
    pattern3 = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (paño|paños)\b', re.IGNORECASE)


    match1 = pattern1.search(texto)
    match2 = pattern2.search(texto)
    match3 = pattern3.search(texto)

    if match1:
        numero_str, _, tipo = match1.groups()
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)

        if tipo.lower() == "diagonales" or tipo.lower() == "diagonal":
            return numero * 2
        elif tipo.lower() == "posiciones" or tipo.lower() == "posición":
            return numero
    
    elif match2:
        numero_str, tipo = match2.groups()
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)
        return f"al menos {numero} celda(s) para alimentadores"


        
    elif match3:
        numero_str = match3.group(1)
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)
        return f"al menos {numero} paño(s)"

    else:
        return False


casos_faltantes = 0
for texto in textos:
    n_pos = extraer_numero_posiciones_v3(texto)
    if n_pos:
        print(n_pos)
    else:
        print("No se encontró el número de posiciones")
        print(texto)
        casos_faltantes += 1

print("casos faltantes: ", casos_faltantes)

In [82]:
def extraer_conexiones(texto):

    def limpiar_conexion(conexion):
    # Eliminar artículos y preposiciones iniciales
        conexion = re.sub(r'^(el |la |los |las |del |de |a |en |para |la conexión de |la construcción de |del |un |una |)', '', conexion, flags=re.IGNORECASE).strip()
        return conexion
    
    try:
        patron_conexiones_inicio = re.compile(r'\bde manera de permitir la conexión\b(.*?)(\.\s|$)', re.IGNORECASE | re.DOTALL)
        patron_alternativo = re.compile(r'\bpaños para (alimentador|alimentadores)\b(.*?)(\.\s|$)', re.IGNORECASE | re.DOTALL)




        match_conexiones_inicio = patron_conexiones_inicio.search(texto)
        match_alternativo = patron_alternativo.search(texto)

        if match_conexiones_inicio:
            
            conexiones = match_conexiones_inicio.group(1).replace("y la", ",").split(",")
            lista_conexiones = [limpiar_conexion(conexion.strip()) for conexion in conexiones]
            return lista_conexiones

        elif match_alternativo:
            conexiones = match_alternativo.group(0).split(",")
            lista_conexiones = [limpiar_conexion(conexion.strip()) for conexion in conexiones]
            return lista_conexiones




    except Exception as e:
        print(f"Error extraccion conexiones: {e}")


    return "Revisar manualmente, al parecer no hay conexiones en el texto."

casos_faltantes = 0
for texto in textos:
    conexiones = extraer_conexiones(texto)
    print(conexiones)


['la obra “Nueva S/E Alto Molle y nueva línea 2x110 kV Alto Molle – Cóndores”.']
['nuevos proyectos en la zona.']
['nuevos proyectos en la zona.']
['nuevos proyectos en la zona.']
['nuevos proyectos en la zona.']
['nuevos proyectos en la zona.']
['la obra “Nueva S/E La Chimba y nueva línea 2x110 kV Mejillones – La Chimba”.']
['la obra “Conexión de línea 1x110 kV Tap Desalant – Desalant en S/E La Portada”.']
['la obra “Nueva S/E Caracoles', 'nuevas líneas 2x220 kV Caracoles – Liqcau y 2x110 kV Guardiamarina – Caracoles”.']
['banco de autotransformadores 220/110 kV existente', 'conexión del paño asociado al proyecto “Extensión de línea 1x220 kV Atacama – Esmeralda y cambio de conexión a S/E Caracoles”', 'conexión de la línea asociada al proyecto “Nueva línea 1x220 kV Esmeralda – Liqcau”', 'conexión de nuevos proyectos en la zona']
['la obra “Nueva S/E Caracoles', 'nuevas líneas 2x220 kV Caracoles – Liqcau y 2x110 kV Guardiamarina – Caracoles”', 'conexión de la obra “Nueva línea 1x220 kV 

In [ ]:
texto = "Adicionalmente, el proyecto considera la construcción de una nueva sala de celdas de 12 kV, en configuración barra principal más barra auxiliar, contemplándose la construcción de, al menos, una celda para alimentador, dos espacios para celdas de futuros alimentadores, las celdas de  conexión del transformador antes mencionado a la barra principal y a la barra auxiliar, la construcción de una celda para equipos de medida y la construcción de celdas para la interconexión con las barras de media tensión existentes."

n_pos = extraer_numero_posiciones_v3(texto)

if n_pos:
    print(n_pos)
else:
    print("No se encontró el número de posiciones")
    print(texto)